In [1]:
# import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics

In [2]:
# read in the data

house = pd.read_csv('../datasets/train.csv')
print(house.shape)
house.head()

(2051, 81)


,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
0,109,533352170,60,RL,NaN,13517,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,130500
1,544,531379050,60,RL,43.0,11492,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,4,2009,WD,220000
2,153,535304180,20,RL,68.0,7922,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,1,2010,WD,109000
3,318,916386060,60,RL,73.0,9802,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,174000
4,255,906425045,50,RL,82.0,14235,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,3,2010,WD,138500


In [3]:
# check for missing values

house.isnull().sum().sort_values(ascending = False)

Pool QC         2042
Misc Feature    1986
Alley           1911
Fence           1651
Fireplace Qu    1000
                ... 
Electrical         0
Central Air        0
Heating QC         0
Sale Type          0
Id                 0
Length: 81, dtype: int64

In [4]:
# check the pct of missingness - prep for dropping values

missing = pd.DataFrame({
                        'total': house.isnull().sum(),
                        'pct': house.isnull().sum()/2051
})

In [5]:
# save the missingness dataset to csv file

missing.sort_values(by = 'total', ascending = False)
missing.to_csv('../datasets/missing.csv', index = False)

# pool qc. misc feature, alley, and fence have mostly missing (over 80%). will drop these columns.

# need to fill 0, none, or median, depending on the variables.

# columns are - 

# lot frontage --> median
# bsmt full/half bath, garage cars/area, bsmt fin sf 1/fin sf 2/unf sf, mas vnr area --> 0
# fire place qu, garage type/finish/qual/cond, bsmt qual/cond/exposure/fin type1/fin type 2, mas vnr type --> none

In [6]:
# check for missing values that are related to garage

house.loc[(house['Garage Type'].isnull() == True) | (house['Garage Yr Blt'].isnull() == True) | (house['Garage Finish'].isnull() == True) | (house['Garage Cars'].isnull() == True) | 
          (house['Garage Area'].isnull() == True) | (house['Garage Qual'].isnull() == True) | (house['Garage Cond'].isnull() == True)][['Garage Type', 'Garage Yr Blt', 'Garage Finish', 'Garage Cars', 'Garage Area', 'Garage Qual', 'Garage Cond']]

,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond
28,NaN,NaN,NaN,0.0,0.0,NaN,NaN
53,NaN,NaN,NaN,0.0,0.0,NaN,NaN
65,NaN,NaN,NaN,0.0,0.0,NaN,NaN
79,NaN,NaN,NaN,0.0,0.0,NaN,NaN
101,NaN,NaN,NaN,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...
1991,NaN,NaN,NaN,0.0,0.0,NaN,NaN
2010,NaN,NaN,NaN,0.0,0.0,NaN,NaN
2027,NaN,NaN,NaN,0.0,0.0,NaN,NaN
2039,NaN,NaN,NaN,0.0,0.0,NaN,NaN


In [7]:
# drop columns that contain over 80% values

house.drop(columns = ['Pool QC', 'Misc Feature', 'Alley', 'Fence'], axis = 1, inplace = True)

In [8]:
# fill missing values with 0, NA, or None

for col in house.columns:
    if col in ['Bsmt Full Bath', 'Bsmt Half Bath', 'Garage Cars', 'Garage Area', 'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Mas Vnr Area', 'Garage Yr Blt', 'Total Bsmt SF']:
        house[col].fillna(0, inplace = True)
    elif col in ['Fireplace Qu', 'Garage Type', 'Garage Finish', 'Garage Qual', 'Garage Cond', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2']:
        house[col].fillna('NA', inplace = True)
    elif col in ['Mas Vnr Type']:
        house[col].fillna('None', inplace = True)
    else:
        pass

In [9]:
# check for missing values again

house.isnull().sum().sort_values(ascending = False)

Lot Frontage     330
SalePrice          0
Exter Cond         0
Roof Style         0
Roof Matl          0
                ... 
Bedroom AbvGr      0
Kitchen AbvGr      0
Kitchen Qual       0
TotRms AbvGrd      0
Id                 0
Length: 77, dtype: int64

In [10]:
# filling with median

house['Lot Frontage'] = house.groupby('Neighborhood')['Lot Frontage'].apply(lambda x: x.fillna(x.median())) 
# --> still 3 nan values, will fill it with median lot size for the ground living area

house['Lot Frontage'] = house.groupby('Gr Liv Area')['Lot Frontage'].apply(lambda x: x.fillna(x.median())) 
# --> still 1 nan value, will fill it with median for ground living area between 1290 and 1300

house.loc[(house['Gr Liv Area'] < 1300) & (house['Gr Liv Area'] > 1290)][['Lot Frontage', 'Neighborhood', 'Gr Liv Area']].sort_values(by = 'Lot Frontage') 
# --> replace NaN with 67.0

house['Lot Frontage'].fillna(67.0, inplace = True)

In [11]:
# only keep residential tags

house['MS Zoning'].value_counts()

house = house[house['MS Zoning'].isin(['RH', 'RL', 'RP', 'RM'])].reset_index(drop = True)

house.shape

(1928, 77)

In [12]:
# check the sq ft information

house[['1st Flr SF', '2nd Flr SF', 'Gr Liv Area', 'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF']]

,1st Flr SF,2nd Flr SF,Gr Liv Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF
0,725,754,1479,533.0,0.0,192.0,725.0
1,913,1209,2122,637.0,0.0,276.0,913.0
2,1057,0,1057,731.0,0.0,326.0,1057.0
3,744,700,1444,0.0,0.0,384.0,384.0
4,831,614,1445,0.0,0.0,676.0,676.0
...,...,...,...,...,...,...,...
1923,1728,0,1728,1011.0,0.0,873.0,1884.0
1924,861,0,861,262.0,0.0,599.0,861.0
1925,1172,741,1913,0.0,0.0,896.0,896.0
1926,1200,0,1200,155.0,750.0,295.0,1200.0


In [13]:
# change the column names for me to type better

house.columns = house.columns.str.replace(' ', '_')
house.columns = house.columns.str.lower()

In [14]:
# make sure if the null values are all filled

house['total_bsmt_sf'] = house['total_bsmt_sf'].fillna(0)
house['total_sf'] = house['gr_liv_area'] + house['total_bsmt_sf']
house.loc[house['total_sf'].isnull() == True][['gr_liv_area', 'total_bsmt_sf', 'total_sf']]

,gr_liv_area,total_bsmt_sf,total_sf


In [15]:
# check columns if everything looks good

house.columns

Index(['id', 'pid', 'ms_subclass', 'ms_zoning', 'lot_frontage', 'lot_area',
       'street', 'lot_shape', 'land_contour', 'utilities', 'lot_config',
       'land_slope', 'neighborhood', 'condition_1', 'condition_2', 'bldg_type',
       'house_style', 'overall_qual', 'overall_cond', 'year_built',
       'year_remod/add', 'roof_style', 'roof_matl', 'exterior_1st',
       'exterior_2nd', 'mas_vnr_type', 'mas_vnr_area', 'exter_qual',
       'exter_cond', 'foundation', 'bsmt_qual', 'bsmt_cond', 'bsmt_exposure',
       'bsmtfin_type_1', 'bsmtfin_sf_1', 'bsmtfin_type_2', 'bsmtfin_sf_2',
       'bsmt_unf_sf', 'total_bsmt_sf', 'heating', 'heating_qc', 'central_air',
       'electrical', '1st_flr_sf', '2nd_flr_sf', 'low_qual_fin_sf',
       'gr_liv_area', 'bsmt_full_bath', 'bsmt_half_bath', 'full_bath',
       'half_bath', 'bedroom_abvgr', 'kitchen_abvgr', 'kitchen_qual',
       'totrms_abvgrd', 'functional', 'fireplaces', 'fireplace_qu',
       'garage_type', 'garage_yr_blt', 'garage_finish', '

In [16]:
# check the data

house.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1928 entries, 0 to 1927
Data columns (total 78 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               1928 non-null   int64  
 1   pid              1928 non-null   int64  
 2   ms_subclass      1928 non-null   int64  
 3   ms_zoning        1928 non-null   object 
 4   lot_frontage     1928 non-null   float64
 5   lot_area         1928 non-null   int64  
 6   street           1928 non-null   object 
 7   lot_shape        1928 non-null   object 
 8   land_contour     1928 non-null   object 
 9   utilities        1928 non-null   object 
 10  lot_config       1928 non-null   object 
 11  land_slope       1928 non-null   object 
 12  neighborhood     1928 non-null   object 
 13  condition_1      1928 non-null   object 
 14  condition_2      1928 non-null   object 
 15  bldg_type        1928 non-null   object 
 16  house_style      1928 non-null   object 
 17  overall_qual  

In [17]:
# follow above steps for the test dataset as well
# read in test data

test = pd.read_csv('../datasets/test.csv')

In [18]:
print(test.shape)
test.isnull().sum().sort_values(ascending = False)

(878, 80)


Pool QC         874
Misc Feature    837
Alley           820
Fence           706
Fireplace Qu    422
               ... 
Heating QC        0
Heating           0
Yr Sold           0
Bsmt Unf SF       0
Id                0
Length: 80, dtype: int64

In [19]:
# drop columns

test.drop(columns = ['Pool QC', 'Misc Feature', 'Fence', 'Alley'], axis = 1, inplace = True)

In [20]:
test.shape

(878, 76)

In [21]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 76 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               878 non-null    int64  
 1   PID              878 non-null    int64  
 2   MS SubClass      878 non-null    int64  
 3   MS Zoning        878 non-null    object 
 4   Lot Frontage     718 non-null    float64
 5   Lot Area         878 non-null    int64  
 6   Street           878 non-null    object 
 7   Lot Shape        878 non-null    object 
 8   Land Contour     878 non-null    object 
 9   Utilities        878 non-null    object 
 10  Lot Config       878 non-null    object 
 11  Land Slope       878 non-null    object 
 12  Neighborhood     878 non-null    object 
 13  Condition 1      878 non-null    object 
 14  Condition 2      878 non-null    object 
 15  Bldg Type        878 non-null    object 
 16  House Style      878 non-null    object 
 17  Overall Qual    

In [22]:
# fill with 0, NA, or None

for col in test.columns:
    if col in ['Bsmt Full Bath', 'Bsmt Half Bath', 'Garage Cars', 'Garage Area', 'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Mas Vnr Area', 'Garage Yr Blt', 'Total Bsmt SF']:
        test[col].fillna(0, inplace = True)
    elif col in ['Fireplace Qu', 'Garage Type', 'Garage Finish', 'Garage Qual', 'Garage Cond', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2']:
        test[col].fillna('NA', inplace = True)
    elif col in ['Mas Vnr Type']:
        test[col].fillna('None', inplace = True)
    else:
        pass

In [23]:
test.isnull().sum().sort_values(ascending = False)

Lot Frontage     160
Electrical         1
Sale Type          0
Exter Qual         0
Roof Style         0
                ... 
Bedroom AbvGr      0
Kitchen AbvGr      0
Kitchen Qual       0
TotRms AbvGrd      0
Id                 0
Length: 76, dtype: int64

In [24]:
# fill with median

test['Lot Frontage'] = test.groupby('Neighborhood')['Lot Frontage'].apply(lambda x: x.fillna(x.median())) 

In [25]:
test['Electrical'].mode()

0    SBrkr
dtype: object

In [26]:
# fill with mode

test.loc[test['Electrical'].isnull() == True][['Id', 'Electrical']]
test['Electrical'].replace(to_replace = np.NaN, value = 'SBrkr', inplace = True)

In [27]:
test.shape

(878, 76)

In [28]:
test[['1st Flr SF', '2nd Flr SF', 'Gr Liv Area', 'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF']]

,1st Flr SF,2nd Flr SF,Gr Liv Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF
0,908,1020,1928,0,0,1020,1020
1,1967,0,1967,0,0,1967,1967
2,664,832,1496,554,0,100,654
3,968,0,968,0,0,968,968
4,1394,0,1394,609,0,785,1394
...,...,...,...,...,...,...,...
873,1084,793,1877,931,153,0,1084
874,1104,884,1988,575,0,529,1104
875,1211,0,1211,250,492,210,952
876,864,0,864,616,0,248,864


In [29]:
# change the column names - no uppercase, no space

test.columns = test.columns.str.replace(' ', '_')
test.columns = test.columns.str.lower()

In [30]:
# take care of the null values

test['total_bsmt_sf'] = test['total_bsmt_sf'].fillna(0)
test['total_sf'] = test['gr_liv_area'] + test['total_bsmt_sf']
test.loc[test['total_sf'].isnull() == True][['gr_liv_area', 'total_bsmt_sf', 'total_sf']]

,gr_liv_area,total_bsmt_sf,total_sf


In [31]:
test.head()

,id,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,lot_shape,land_contour,utilities,...,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,misc_val,mo_sold,yr_sold,sale_type,total_sf
0,2658,902301120,190,RM,69.0,9142,Pave,Reg,Lvl,AllPub,...,60,112,0,0,0,0,4,2006,WD,2948
1,2718,905108090,90,RL,75.0,9662,Pave,IR1,Lvl,AllPub,...,0,0,0,0,0,0,8,2006,WD,3934
2,2414,528218130,60,RL,58.0,17104,Pave,IR1,Lvl,AllPub,...,24,0,0,0,0,0,9,2006,New,2150
3,1989,902207150,30,RM,60.0,8520,Pave,Reg,Lvl,AllPub,...,0,184,0,0,0,0,7,2007,WD,1936
4,625,535105100,20,RL,74.5,9500,Pave,IR1,Lvl,AllPub,...,76,0,0,185,0,0,7,2009,WD,2788


In [32]:
# use id column as index

house.set_index('id', inplace = True)
test.set_index('id', inplace = True)

In [33]:
house.head(2)

,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,lot_shape,land_contour,utilities,lot_config,...,enclosed_porch,3ssn_porch,screen_porch,pool_area,misc_val,mo_sold,yr_sold,sale_type,saleprice,total_sf
id,,,,,,,,,,,,,,,,,,,,,
109,533352170,60,RL,70.0,13517,Pave,IR1,Lvl,AllPub,CulDSac,...,0,0,0,0,0,3,2010,WD,130500,2204.0
544,531379050,60,RL,43.0,11492,Pave,IR1,Lvl,AllPub,CulDSac,...,0,0,0,0,0,4,2009,WD,220000,3035.0


In [34]:
test.head(2)

,pid,ms_subclass,ms_zoning,lot_frontage,lot_area,street,lot_shape,land_contour,utilities,lot_config,...,open_porch_sf,enclosed_porch,3ssn_porch,screen_porch,pool_area,misc_val,mo_sold,yr_sold,sale_type,total_sf
id,,,,,,,,,,,,,,,,,,,,,
2658,902301120,190,RM,69.0,9142,Pave,Reg,Lvl,AllPub,Inside,...,60,112,0,0,0,0,4,2006,WD,2948
2718,905108090,90,RL,75.0,9662,Pave,IR1,Lvl,AllPub,Inside,...,0,0,0,0,0,0,8,2006,WD,3934


In [36]:
# store datasets as csv

house.to_csv('../datasets/train_clean.csv')
test.to_csv('../datasets/test_clean.csv')